In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
path = 'dataset/'
X_train = np.load(path+'X_train_1d.npy')
X_test = np.load(path+'X_test_1d.npy')
X_val = np.load(path+'X_val_1d.npy') #this may not be required for now

t_train = np.load(path+'y_grain_cond_ka_train.npy')
t_train = np.where(t_train>=0, 1, 0)
t_test = np.load(path+'y_grain_cond_ka_test.npy')
t_test = np.where(t_test>=0, 1, 0)
t_val = np.load(path+'y_grain_cond_ka_val.npy') #this may not be required for now
t_val = np.where(t_val>=0, 1, 0)

del_idx_train = np.unique(np.where(np.isnan(X_train))[0])
del_idx_test = np.unique(np.where(np.isnan(X_test))[0])
del_idx_val = np.unique(np.where(np.isnan(X_val))[0])

X_train = np.delete(X_train, del_idx_train, axis=0)
t_train = np.delete(t_train, del_idx_train, axis=0)

X_test = np.delete(X_test, del_idx_test, axis=0)
t_test = np.delete(t_test, del_idx_test, axis=0)

X_val = np.delete(X_val, del_idx_val, axis=0)
t_val = np.delete(t_val, del_idx_val, axis=0)

In [5]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [6]:
def create_bnn_model(train_size, hidden_dim, act_fn):
    inputs = layers.Input(shape=(4,))
    features = layers.BatchNormalization()(inputs)
    features = tfp.layers.DenseVariational(units=hidden_dim, make_prior_fn=prior, make_posterior_fn=posterior, kl_weight=1/train_size, activation=act_fn)(features)
    features = tfp.layers.DenseVariational(units=hidden_dim, make_prior_fn=prior, make_posterior_fn=posterior, kl_weight=1/train_size, activation=act_fn)(features)
    features = tfp.layers.DenseVariational(units=hidden_dim, make_prior_fn=prior, make_posterior_fn=posterior, kl_weight=1/train_size, activation=act_fn)(features)
    features = tfp.layers.DenseVariational(units=hidden_dim, make_prior_fn=prior, make_posterior_fn=posterior, kl_weight=1/train_size, activation=act_fn)(features)    
    outputs = layers.Dense(units=1, activation='sigmoid')(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [7]:
def run_experiment(model, loss, X_train, t_train, X_test, t_test, lr):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=loss,
        metrics=['accuracy'],
    )

    print("Start training the model...")
    history = model.fit(X_train, t_train, epochs=num_epochs, validation_data=(X_test, t_test), batch_size=64)
    print("Model training finished.")
    _, acc1 = model.evaluate(X_train, t_train, verbose=0)
    print(f"Train accuracy: {round(acc1, 3)}")

    print("Evaluating model performance...")
    _, acc2 = model.evaluate(X_test, t_test, verbose=0)
    print(f"Test accuracy: {round(acc2, 3)}")
    return history, [acc1, acc2]

In [9]:
lr_list = [0.01, 1e-5]
num_epochs = 20
bce_loss = tf.keras.losses.BinaryCrossentropy()

history_list = []
acc = []
act_fn_list = ['relu', 'sigmoid', 'tanh']
for act_fn in act_fn_list:
    dim_list = [5, 10, 20]
    for hidden in dim_list: 
        for lr in lr_list:
            bnn_model_full = create_bnn_model(len(X_train), hidden, act_fn)
            history_temp, acc_temp = run_experiment(bnn_model_full, bce_loss, X_train, t_train, X_test, t_test, lr)
            history_list.append(history_temp)
            acc.append(acc_temp)



Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.
Start training the model...
Epoch 1/20
33/33 [==============================] - 4s 39ms/step - loss: 1.0152 - accuracy: 0.4913 - val_loss: 1.1029 - val_accuracy: 0.4731
Epoch 2/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7894 - accuracy: 0.5019 - val_loss: 0.7042 - val_accuracy: 0.6677
Epoch 3/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7391 - accuracy: 0.5468 - val_loss: 0.6287 - val_accuracy: 0.7500
Epoch 4/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7586 - accuracy: 0.5463 - val_loss: 0.6823 - val_accuracy: 0.5934
Epoch 5/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7079 - accuracy: 0.5632 - val_loss: 0.6338 - val_accuracy: 0.7199
Epoch 6/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7066 - accuracy: 0.5516 - val_loss: 0.6347 - val_accuracy: 0.7152
Epoch 7

33/33 [==============================] - 0s 5ms/step - loss: 0.7565 - accuracy: 0.5689 - val_loss: 0.6924 - val_accuracy: 0.6883
Epoch 15/20
33/33 [==============================] - 0s 5ms/step - loss: 0.7633 - accuracy: 0.5771 - val_loss: 0.6694 - val_accuracy: 0.7389
Epoch 16/20
33/33 [==============================] - 0s 4ms/step - loss: 0.7542 - accuracy: 0.5805 - val_loss: 0.7103 - val_accuracy: 0.7389
Epoch 17/20
33/33 [==============================] - 0s 4ms/step - loss: 0.7522 - accuracy: 0.5800 - val_loss: 0.7027 - val_accuracy: 0.7389
Epoch 18/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7530 - accuracy: 0.5776 - val_loss: 0.6900 - val_accuracy: 0.7389
Epoch 19/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7505 - accuracy: 0.5771 - val_loss: 0.7098 - val_accuracy: 0.7389
Epoch 20/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7518 - accuracy: 0.5805 - val_loss: 0.7156 - val_accuracy: 0.7389
Model training finished.


33/33 [==============================] - 0s 12ms/step - loss: 6.5780 - accuracy: 0.5174 - val_loss: 3.1955 - val_accuracy: 0.4937
Epoch 10/20
33/33 [==============================] - 0s 12ms/step - loss: 6.2326 - accuracy: 0.5270 - val_loss: 3.9391 - val_accuracy: 0.5491
Epoch 11/20
33/33 [==============================] - 0s 11ms/step - loss: 5.2652 - accuracy: 0.5188 - val_loss: 4.2745 - val_accuracy: 0.5285
Epoch 12/20
33/33 [==============================] - 0s 11ms/step - loss: 4.5820 - accuracy: 0.5058 - val_loss: 3.6584 - val_accuracy: 0.6234
Epoch 13/20
33/33 [==============================] - 0s 11ms/step - loss: 5.7851 - accuracy: 0.5159 - val_loss: 3.5467 - val_accuracy: 0.6582
Epoch 14/20
33/33 [==============================] - 0s 11ms/step - loss: 6.2321 - accuracy: 0.5164 - val_loss: 6.5707 - val_accuracy: 0.5237
Epoch 15/20
33/33 [==============================] - 0s 11ms/step - loss: 6.8912 - accuracy: 0.4875 - val_loss: 4.9686 - val_accuracy: 0.5902
Epoch 16/20
33/33 

33/33 [==============================] - 0s 4ms/step - loss: 0.7193 - accuracy: 0.5632 - val_loss: 0.6566 - val_accuracy: 0.7278
Epoch 5/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7122 - accuracy: 0.5796 - val_loss: 0.6881 - val_accuracy: 0.6171
Epoch 6/20
33/33 [==============================] - 0s 4ms/step - loss: 0.7119 - accuracy: 0.5757 - val_loss: 0.6502 - val_accuracy: 0.7389
Epoch 7/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7113 - accuracy: 0.5743 - val_loss: 0.6429 - val_accuracy: 0.7389
Epoch 8/20
33/33 [==============================] - 0s 3ms/step - loss: 0.7053 - accuracy: 0.5805 - val_loss: 0.6530 - val_accuracy: 0.7389
Epoch 9/20
33/33 [==============================] - 0s 4ms/step - loss: 0.7075 - accuracy: 0.5805 - val_loss: 0.6678 - val_accuracy: 0.7389
Epoch 10/20
33/33 [==============================] - 0s 4ms/step - loss: 0.7052 - accuracy: 0.5805 - val_loss: 0.6381 - val_accuracy: 0.7389
Epoch 11/20
33/33 [===========

33/33 [==============================] - 0s 12ms/step - loss: 1.1777 - accuracy: 0.5805 - val_loss: 1.0969 - val_accuracy: 0.7389
Model training finished.
Train accuracy: 0.581
Evaluating model performance...
Test accuracy: 0.739
Start training the model...
Epoch 1/20
33/33 [==============================] - 5s 49ms/step - loss: 0.8691 - accuracy: 0.4436 - val_loss: 0.9678 - val_accuracy: 0.3782
Epoch 2/20
33/33 [==============================] - 0s 14ms/step - loss: 0.8772 - accuracy: 0.4248 - val_loss: 1.0404 - val_accuracy: 0.3623
Epoch 3/20
33/33 [==============================] - 0s 12ms/step - loss: 0.8718 - accuracy: 0.4436 - val_loss: 1.0400 - val_accuracy: 0.2785
Epoch 4/20
33/33 [==============================] - 0s 12ms/step - loss: 0.8669 - accuracy: 0.4306 - val_loss: 0.9635 - val_accuracy: 0.2658
Epoch 5/20
33/33 [==============================] - 0s 11ms/step - loss: 0.8871 - accuracy: 0.4368 - val_loss: 1.0257 - val_accuracy: 0.2611
Epoch 6/20
33/33 [===================

33/33 [==============================] - 0s 2ms/step - loss: 0.8289 - accuracy: 0.4990 - val_loss: 0.8359 - val_accuracy: 0.5142
Epoch 16/20
33/33 [==============================] - 0s 3ms/step - loss: 0.8635 - accuracy: 0.4942 - val_loss: 1.0117 - val_accuracy: 0.3101
Epoch 17/20
33/33 [==============================] - 0s 2ms/step - loss: 0.8730 - accuracy: 0.5019 - val_loss: 0.8982 - val_accuracy: 0.5079
Epoch 18/20
33/33 [==============================] - 0s 3ms/step - loss: 0.8481 - accuracy: 0.5082 - val_loss: 0.8739 - val_accuracy: 0.4842
Epoch 19/20
33/33 [==============================] - 0s 2ms/step - loss: 0.8313 - accuracy: 0.4932 - val_loss: 0.6837 - val_accuracy: 0.6266
Epoch 20/20
33/33 [==============================] - 0s 3ms/step - loss: 0.8638 - accuracy: 0.4769 - val_loss: 0.6746 - val_accuracy: 0.6535
Model training finished.
Train accuracy: 0.492
Evaluating model performance...
Test accuracy: 0.46
Start training the model...
Epoch 1/20
33/33 [=====================

33/33 [==============================] - 0s 12ms/step - loss: 1.1117 - accuracy: 0.5800 - val_loss: 1.0305 - val_accuracy: 0.7389
Epoch 11/20
33/33 [==============================] - 0s 12ms/step - loss: 1.0891 - accuracy: 0.5805 - val_loss: 1.0248 - val_accuracy: 0.7231
Epoch 12/20
33/33 [==============================] - 0s 11ms/step - loss: 1.0801 - accuracy: 0.5829 - val_loss: 1.0448 - val_accuracy: 0.7231
Epoch 13/20
33/33 [==============================] - 0s 11ms/step - loss: 1.0878 - accuracy: 0.5728 - val_loss: 1.0578 - val_accuracy: 0.7247
Epoch 14/20
33/33 [==============================] - 0s 11ms/step - loss: 1.0930 - accuracy: 0.5704 - val_loss: 1.0382 - val_accuracy: 0.7389
Epoch 15/20
33/33 [==============================] - 0s 11ms/step - loss: 1.1086 - accuracy: 0.5786 - val_loss: 1.0766 - val_accuracy: 0.6994
Epoch 16/20
33/33 [==============================] - 0s 11ms/step - loss: 1.1214 - accuracy: 0.5733 - val_loss: 1.0641 - val_accuracy: 0.7389
Epoch 17/20
33/33 

In [23]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(X_test).numpy())
    predicted = np.concatenate(predicted, axis=1)
#     print(predicted.shape)
    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()
    for idx in range(len(prediction_mean)):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {t_test[idx]}"
        )


compute_predictions(bnn_model_full)

Predictions mean: 0.52, min: 0.07, max: 0.94, range: 0.87 - Actual: 0
Predictions mean: 0.52, min: 0.07, max: 0.94, range: 0.87 - Actual: 0
Predictions mean: 0.52, min: 0.07, max: 0.94, range: 0.87 - Actual: 0
Predictions mean: 0.52, min: 0.07, max: 0.94, range: 0.87 - Actual: 0
Predictions mean: 0.51, min: 0.07, max: 0.93, range: 0.85 - Actual: 0
Predictions mean: 0.51, min: 0.07, max: 0.93, range: 0.85 - Actual: 0
Predictions mean: 0.51, min: 0.07, max: 0.93, range: 0.85 - Actual: 0
Predictions mean: 0.51, min: 0.07, max: 0.93, range: 0.85 - Actual: 0
Predictions mean: 0.51, min: 0.07, max: 0.93, range: 0.85 - Actual: 0
Predictions mean: 0.57, min: 0.08, max: 0.95, range: 0.87 - Actual: 0
Predictions mean: 0.47, min: 0.05, max: 0.95, range: 0.9 - Actual: 0
Predictions mean: 0.47, min: 0.05, max: 0.95, range: 0.9 - Actual: 0
Predictions mean: 0.47, min: 0.05, max: 0.95, range: 0.9 - Actual: 0
Predictions mean: 0.47, min: 0.05, max: 0.95, range: 0.9 - Actual: 0
Predictions mean: 0.47, 